In [1]:
import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *
import subprocess
import sqlite3
import shutil
import plotly.express as px
from datetime import datetime as dt, timedelta
import pickle

In [2]:

for m in master_list:
  
    model_area = m[0]
    model = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    element_filter = m[7]
    wwtp_pipe = m[9]
    outfall_summary = m[10]
    mus_path = m[11]
    custom_plot_dict = m[12]
    
    plot_specs = pd.read_csv(custom_plot_dict['specs'],dtype={'MUID':'str'})
    
    has_axis2 = True if custom_plot_dict['axis2_title'] != 0 else False
   
    outfall_disconnection_df = pd.read_csv(outfall_summary,dtype={'Structure':'str'})
    
    if not os.path.isdir(output_folder + r'\Maps_And_CSS'): os.makedirs(output_folder + r'\Maps_And_CSS') 
    shutil.copy2('style_mb.css', output_folder + r'\Maps_And_CSS\style_mb.css')
    shutil.copy2('script_mb.js', output_folder + r'\Maps_And_CSS\script_mb.js')
    
    #find subfolders for MIKE+
    result_dict = {}
    if model[-7:] == '.sqlite':       
        not_founds = []
        for r in result_list:
            file = r[1]
            file_found = False
            for f1 in os.listdir(result_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder
                    result_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                    try:
                        for f2 in os.listdir(result_folder + '\\' + result_subfolder):
                            if os.path.basename(f2) == file:
                                result_dict[file] = [f1,'\\' + result_subfolder]
                                file_found = True
                    except:
                        pass
            if not file_found:
                not_founds.append(file)
                
    else:

        for r in result_list:
            file = r[1]
            run_model = r[4]
            result_dict[file] = [run_model,'']
        
 
    output_subfolder = output_folder + '\\Custom_Plots'
    if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

    html_path = output_subfolder + '\\' + custom_plot_dict['file_name_no_extension'] + '.html'
    f = open(html_path, "w")
    
    f.write('<!DOCTYPE html>\n')
    f.write('<html>\n')
    f.write('<head>\n')
    f.write('<meta charset="utf-8">\n')
    f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style_mb.css">\n')
    f.write('<script src="..\Maps_And_CSS\script_mb.js"></script>\n')  
    f.write('</head>\n')
    f.write('<body>\n\n')

    f.write('<div class="tab">\n')
    for r in result_list:
        tab = r[0]       
        f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    f.write('</div>\n') #f.write('<div class="tab">\n')
           
    for r in result_list:
                        
        header = r[0]
        tab = header
        file = r[1]
        model_path = result_folder + '\\' + result_dict[file][0]
        result_network_path = result_folder + '\\' + result_dict[file][1] + '\\' + file
        res1d = Res1D(result_network_path)
        
        start = res1d.time_index.min()
        end = res1d.time_index.max()
  
        print('Importing ' + header)
        
        f.write('<div id="' + tab + '" class="tabcontent">\n')  
        
        
        if has_axis2:
            fig = make_subplots(specs=[[{"secondary_y": True}]])
        else:
            fig = go.Figure()
                    
        for index, row in plot_specs.iterrows():
            
            muid = row['MUID']
            
            color = px.colors.qualitative.Plotly[index]
            
            has_dfs0 = False
            if not pd.isna(row['DFS0_Path']):
                has_dfs0 = True
                dfs0_df = mikeio.read(row['DFS0_Path']).to_dataframe()
                dfs0_df = dfs0_df[[row['DFS0_Item']]]
                dfs0_df = dfs0_df.loc[(dfs0_df.index >= start) & (dfs0_df.index <= end)]
                if len(dfs0_df) > 0:
                    has_dfs0 = True
            
            use_axis2 = True if row['Axis'] == 2 else False
            
            if row['Layer'].lower() == 'msm_node':
                values = list(res1d.query.GetNodeValues(muid, row['Quantity']))
            else:
                if row['Layer'].lower() == 'msm_link':             
                    values = list(res1d.query.GetReachEndValues(muid, row['Quantity']))
                else:
                    values = list(res1d.query.GetReachEndValues(row['Layer'][4:] + ':' + muid, row['Quantity']))
                if custom_plot_dict['discharge_multiplier'] != 1:
                    values = [k * custom_plot_dict['discharge_multiplier'] for k in values]
            
            if has_axis2:
                fig.add_trace(go.Scatter(x=res1d.time_index, 
                                             y = values, 
                                             mode='lines',name=row['Gauge']
                                             ,line=dict(color=color)),secondary_y=use_axis2) 
            else:
                fig.add_trace(go.Scatter(x=res1d.time_index, 
                                             y = values, 
                                             mode='lines',name=row['Gauge'],line=dict(color=color))) 
            
            
            if has_dfs0:
                if has_axis2:
                    fig.add_trace(go.Scatter(x=dfs0_df.index, 
                                                 y = dfs0_df[row['DFS0_Item']], 
                                                 mode='lines',name=row['Gauge'] + ' - Measured', 
                                                 line=dict(color=color, dash='dash')),
                                                 secondary_y=use_axis2)
                else:
                    fig.add_trace(go.Scatter(x=dfs0_df.index, 
                                                 y = dfs0_df[row['DFS0_Item']], 
                                                 mode='lines',name=row['Gauge'] + ' - Measured',
                                                 line=dict(color=color, dash='dash')))               

        fig.update_layout(
            title = custom_plot_dict['plot_title'],
            autosize=False,
            width = 1362,
            height=800,
            margin=dict(
                l=50,
                r=50,
                b=25,
                t=35,
                pad=4
                ),
            yaxis_title = custom_plot_dict['axis1_title'])  
        
        if has_axis2:
            fig['layout']['yaxis2']['title'] = custom_plot_dict['axis2_title']

        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        
        f.write
        
        f.write('</div>\n') #f.write('<div class="main">\n')
        f.write('</div>\n') #f.write('<div id="' + tab + '" class="tabcontent">\n') 
 
    f.write('</body>\n')
    f.write('</html>\n')
    f.close()
                
print('Done')                


Importing 25yr 24h No Climate Change
Done


In [3]:
px.colors.qualitative.Plotly[0]

'#636EFA'